# Naive Bayes

Teorema de Bayes:

In [2]:
%%latex
$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$

<IPython.core.display.Latex object>

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [4]:
df = pd.read_csv("data/naive-db.csv")

In [5]:
df.head()

,label,Amplitude_bin,Amplitude,Std_bin,Std,PeriodLS_bin,PeriodLS,Mean_bin,Mean,MaxSlope_bin,MaxSlope,Meanvariance_bin,Meanvariance,LinearTrend,LinearTrend_bin
0,0,7,0.53300,6,0.283701,5,1.089524,17,19.338182,15,7.039598,3,0.014671,0.000036,11
1,0,6,0.27150,3,0.127947,7,3.209689,18,19.586202,15,10.640067,2,0.006533,-0.000015,10
2,0,6,0.30150,3,0.138996,6,2.268446,18,19.913902,15,21.778886,2,0.006980,-0.000007,11
3,0,5,0.16400,2,0.095606,5,1.561432,9,15.151209,15,0.353493,2,0.006310,0.000001,10
4,0,5,0.11125,2,0.056008,5,1.166648,15,18.361809,15,17.002882,1,0.003050,0.000002,11


In [6]:
df['label'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [7]:
np.random.seed(123)
shuffled = df.loc[np.random.permutation(df.index)]

percentil80 = int(len(df)*0.8)
train = shuffled[:percentil80]
test = shuffled[percentil80:]

len(train), len(test)

(48061, 12016)

Que clase es mas probable dado los datos?

$P(0\ |\ Features) =\ ?$  
$P(1\ |\ Features) =\ ?$  
$P(2\ |\ Features) =\ ?$  
$P(3\ |\ Features) =\ ?$  
$P(4\ |\ Features) =\ ?$  
$P(5\ |\ Features) =\ ?$  

Debemos tomar el $argmax_c\{ P(c\ |\ Features)\}$.

**Por teorema de Bayes:**

$P(L\ |\ x_1, x_2, x_3)\ =\ \frac{P(x_1,\ x_2,\ x_3\ |\ L)\ \cdot\ P(L)}
                                 {P(x_1,\ x_2,\ x_3)}$
                                 
**Si asumimos independencia entre las features, obtenemos:**

$P(L\ |\ x_1, x_2, x_3)\ \propto\ P(L)\ \cdot
                                  P(x_1\ |\ L)\ \cdot
                                  P(x_2\ |\ L)\ \cdot
                                  P(x_3\ |\ L)\ \cdot
                                  P(x_4\ |\ L)\ \cdots$

In [16]:
mu_matrix = []
sigma_matrix = []
for label in train['label'].unique():
    mus_given_L = []
    sigmas_given_L = []
    for feature in train.columns[train.columns != 'label']:
        mus_given_L.append(train[train['label'] == label][feature].mean())
        sigmas_given_L.append(train[train['label'] == label][feature].std())
    mu_matrix.append(mus_given_L)
    sigmas_matrix.append(sigmas_given_L)
    
def p_class(L_value):
    return train['label'][train['label'] == L_value].count() / len(train)

labels_probabilities = [p_class(label) for label in train.columns]


def proportional_class_given_row(L_value, row):
    product = labels_probabilities[list(test.columns).index(L_value)]
    for feature in row.index[row.index != 'label']:
        f_value = row[feature]
        mu = mu_matrix[list(test.columns).index(L_value)]  # # ESTO UEDA PENDIENTE, DEBI HABER HEHO UN DF, NO MATRI
        product *= norm(m(feature, f_value, L_value)
    return product

In [17]:
proportional_class_given_row(0, train.iloc[0])

1.2777322603745798e-109

In [18]:
# Clasificamos una fila cualquiera
correct = 0
wrong = 0
for i in range(len(test)):
    row = test.iloc[i]
    maximo = 0
    for L_value in test['label'].unique():
        p = proportional_class_given_row(L_value, row)
        if p > maximo:
            maximo = p
            arg = L_value
    if arg == test.iloc[i]['label']:
        correct += 1
    else:
        wrong += 1

/home/gregschuit/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: invalid value encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/gregschuit/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)


KeyboardInterrupt: 

In [23]:
list(train.columns).index('Amplitude')

2